# **ETAPA 3 - PROCESSAMENTO PARA CAMADA BRONZE**

<br>

---

<br>

Essa etapa será responsável por mover os dados para a camada bronze, colocando em um formato padrão mas sem mais nenhum refinamento, ainda mantendo os dados brutos.

*`Esse é um modelo genérico, coloque as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO IMPORTANTE: No Databricks Free Edition trabalha-se com catálogos, que é a maneira de na prática aplicar um Data Lakehouse***

<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, 
    year, 
    month, 
    regexp_extract
)
from pyspark.sql.types import (
    StructType, 
    StructField,
    IntegerType, 
    StringType, 
    DoubleType, 
    DateType
)

<br>

---

<br>

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

In [0]:
spark = (
    SparkSession.builder
        .appName("Load Data Bronze")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "128MB")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .getOrCreate()
)

<br>

---

<br>

### Parte 3 - **Definindo Origens e Destinos**

`Insira nas variáveis:` <br>
`--> nome_datalakehouse --> nome do Data Lakehouse ` <br>
`--> nome_camada_origem --> nome da camada de origem dos dados` <br>
`--> nome_volume_origem --> nome do volume de origem dos dados dentro da camada` <br>
`--> nome_camada_bronze --> nome da camada de destino dos dados` <br>
`--> nome_volume_bronze --> nome do volume de destino dos dados dentro da camada` <br>

In [0]:
nome_datalakehouse = "lhdw"
nome_camada_origem = "source"
nome_volume_origem = "vendas"
nome_camada_bronze = "bronze"
nome_volume_bronze = "vendas"

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_origem}/{nome_volume_origem}/"
bronze_destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_bronze}/{nome_volume_bronze}/"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"""CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_bronze}.{nome_volume_bronze}""")

<br>

---

<br>

### Parte 4 - **Leitura dos Dados**

Para que seja possível manipular os dados, antes precisamos ler todos eles já com uma estrutura pré-definida.

O código a seguir cria essa estrutura:

In [0]:
estrutura_schema_dados = StructType([
    StructField("IDProduto", IntegerType(), True),
    StructField("Data", DateType(), True),
    StructField("IDCliente", IntegerType(), True),
    StructField("IDCampanha", IntegerType(), True),
    StructField("Unidades", IntegerType(), True),
    StructField("Produto", StringType(), True),
    StructField("Categoria", StringType(), True),
    StructField("Segmento", StringType(), True),
    StructField("IDFabricante", IntegerType(), True),
    StructField("Fabricante", StringType(), True),
    StructField("CustoUnitario", DoubleType(), True),
    StructField("PrecoUnitario", DoubleType(), True),
    StructField("CodigoPostal", StringType(), True),
    StructField("EmailNome", StringType(), True),
    StructField("Cidade", StringType(), True),
    StructField("Estado", StringType(), True),
    StructField("Regiao", StringType(), True),
    StructField("Distrito", StringType(), True),
    StructField("Pais", StringType(), True)
])

O código a seguir faz a leitura dos dados, adicionando a coluna nome do arquivo durante a leitura (usando de base a estrutura feita no código anterior) e armarzenando o resultado desse processo em um data frame para que seja possível manipular esses dados.

In [0]:
dataframe_dados = (
    spark.read
        .option("header", "true")
        .schema(estrutura_schema_dados)
        .csv(origem_dados)
        .withColumn(
            "filename",
            regexp_extract(col('_metadata.file_path'), "([^/]+)$", 0)
        )
)

<br>

---

<br>

### Parte 5 - **Salvando os Dados no Formato e Local Corretos**

Por fim, precisamos criar o parquet com os dados que importamos e com a estrutura que construimos anteriormente e mover para camada bronze.

O código a seguir faz essa compressão e envio para o caminho certo (já particionando em ano e mês para ser mais otimizado quando precisarmos utilizar esses dados granularmente):

In [0]:
(
    dataframe_dados 
        .withColumn("Ano", year("Data")) 
        .withColumn("Mes", month("Data")) 
        .write.mode("overwrite").partitionBy("Ano", "Mes").parquet(bronze_destino_dados)
)

> O modo de escrita define como os dados serão gravados no destino:
- ***overwrite***: remove os dados existentes no caminho/partições e grava tudo novamente
- ***append***: adiciona novos dados às partições existentes, sem apagar o que já existe 

> Em pipelines produtivos, 'append' é o mais comum para cargas incrementais. 'overwrite' costuma ser usado apenas em reprocessamentos completos ou ambientes de teste

<br> <br>

---

<br>

### Parte 6 - **Limpeza de Cache e Outros**

O código a seguir exclui todos os Data Frames que podem estar em cache no algoritmo:

In [0]:
"""
No Databricks Free Edition (Serverless), o gerenciamento de cache e memória é feito automaticamente pela plataforma. Por esse motivo, alguns comandos tradicionais do Spark não são suportados, como:

- dataframe.unpersist()
- spark.catalog.clearCache()

A alternativa é remover referências aos DataFrames manualmente e forçar a coleta de lixo do Python quando necessário.
"""

del dataframe_dados

import gc
gc.collect()

<br>

---

<br>

### **Resultado Final**

O código a seguir monta um novo Data Frame com os dados diretamente da camada bronze 

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

"""
dataframe_bronze = spark.read.parquet(bronze_destino_dados)

display(dataframe_bronze)
"""

E o código a seguir faz a mesma coisa que o anterior, mas filtrando para mostrar somente mês de janeiro. Mostrando assim a performance mais rápida por ter dividido anteriormente:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

"""
dataframe_bronze_janeiro = (
    spark.read
        .parquet(bronze_destino_dados)
        .filter(
            (col("Ano") == 2012) &
            (col("Mes") == 1)
        )
)

display(dataframe_bronze_janeiro)
"""